In [ ]:
# ESN with bias architecture
class ESN:
    def __init__(self, n_units, in_dim, out_dim, sigma_in=0.5):
        
        self.n_units = n_units # 
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.bias_in = 1.0 # input bias
        self.bias_out = 1.0 # output bias 

        connectivity   = 2

        self.sigma_in = sigma_in # input scaling
        self.rho = 0.9  # spectral radius
        
        sparseness =  1 - connectivity/(n_units-1) 
        
        self.tikh = 1e-4 # Tikhonov factor

        self.Win = np.zeros((in_dim+1, n_units))
        for j in range(self.n_units):
            self.Win[np.random.randint(0, in_dim+1),j] = np.random.uniform(-1, 1) #only one element different from zero per row

        # practical way to set the sparseness
        self.W = np.random.uniform(-1, 1, (n_units, n_units)) * (np.random.rand(n_units, n_units) < (1-sparseness))
        spectral_radius = np.max(np.abs(np.linalg.eigvals(self.W)))

        self.W /= spectral_radius #scaled to have unitary spec radius
        # plt.matshow(self.W, cmap='Greys')

    
    def step(self, r_pre, x):
        """ Advances one ESN time step.
            Args:
                x_pre: reservoir state
                u: input
            Returns:
                new augmented state (new state with bias_out appended)
        """
        # input bias added
        x_augmented = np.hstack([x, self.bias_in]) 
        # hyperparameters are explicit here
        r_post = np.tanh(np.dot(x_augmented*self.sigma_in, self.Win) + self.rho*np.dot(r_pre, self.W)) 
        # output bias added
        r_augmented = np.hstack([r_post, self.bias_out])

        return r_augmented

    def open_loop(self, X, r0):
        """ Advances ESN in open-loop / idle iteration.
            Args:
                X: input time series
                r0: initial reservoir state
            Returns:
                time series of augmented reservoir states
                X[0] to X[N-1] -> R[0] to R[N]
        """
        N = X.shape[0]
        R = np.empty((N + 1, self.n_units+1))
        R[0] = np.hstack([r0, self.bias_out])
        for i in 1 + np.arange(N):
            R[i] = self.step(R[i-1,:self.n_units], X[i-1])

        return R

    def train(self, X_washout, X_train, Y_train):
        """ Train the ESN.
            Args:
                X_washout: idle iteration input time series
                X_train: training input time series
        """
        ## washout phase / idle iteration
        rf_washout = self.open_loop(X_washout, np.zeros(self.n_units))[-1,:self.n_units]

        ## open-loop train phase
        R = self.open_loop(X_train, rf_washout)

        ## Ridge Regression
        LHS  = np.dot(R[1:].T, R[1:]) + self.tikh*np.eye(self.n_units+1)
        RHS  = np.dot(R[1:].T, Y_train)
        self.Wout = np.linalg.solve(LHS, RHS)


    def evolve(self, r0, N_evo):
        """Advances ESN in closed-loop
            Args:
                    r0: initial reservoir state
                    N_evo: Number of iterations
                Returns:
                    time series of x_CL
                    R[0] -> X[0] to X[N_evo]
        """
        R = np.empty((N_evo + 1, self.n_units+1))
        R[0] = r0
        
        Yh = np.zeros((N_evo+1, self.out_dim))
        Yh[0] = np.dot(R[0], self.Wout)
        
        for i in 1 + np.arange(N_evo):
            R[i] = self.step(R[i-1,:self.n_units], Yh[i-1])
            Yh[i] = np.dot(R[i], self.Wout)
            
        return Yh
    
    def evolve_edge_removal(self, r0, j, i, N_evo):
        """Advances ESN in Intervened-loop
            Args:
                    r0: initial reservoir state
                    N_evo: Number of iterations
                    j: dim index of the cause
                    i: dim index of the effect
                Returns:
                    time series of x_j->i
                    R[0] -> X[0] to X[N_evo]
        """
        R = np.empty((N_evo + 1, self.n_units+1))
        Rprime = np.empty((N_evo + 1, self.n_units+1))
        R[0] = r0
        Rprime[0] = r0

        Yh = np.zeros((N_evo+1, self.out_dim))
        Yh[0] = np.dot(r0, self.Wout)

        Ain = np.ones(self.in_dim) # 0_j
        Ain[j] = 0

        Aout = np.ones(self.out_dim) # 0_i
        Aout[i] = 0

        Aoutprime = np.zeros(self.out_dim) # 1_i
        Aoutprime[i] = 1

        for i in 1 + np.arange(N_evo):
            R     [i] = self.step(R     [i-1,:self.n_units], Yh[i-1])
            Rprime[i] = self.step(Rprime[i-1,:self.n_units], Yh[i-1] * Ain)
            Yh[i] = np.dot(R[i], self.Wout) * Aout + np.dot(Rprime[i], self.Wout) * Aoutprime

        return Yh
    